In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd

In [2]:
data = pd.read_csv("filtered_data.csv")

In [3]:
pd.set_option('display.max_colwidth', None)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3743 non-null   int64 
 1   Result      3743 non-null   object
 2   Text        3743 non-null   object
dtypes: int64(1), object(2)
memory usage: 87.9+ KB


In [12]:
data = data.drop("Unnamed: 0" , axis=1 )
data.head()


,Result,Text
0,yes,I've always been very hard on myself to be thi...
1,yes,"I struggle a lot with motivation, to do someth..."
2,yes,diagnosed with dysthymia a few months ago. psy...
3,yes,I keep cycling randomly through the same half ...
4,yes,I’ve been feeling actively bad for the past fe...


In [13]:
data = data[data['Text'].apply(lambda x: isinstance(x, str))]

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Result  3743 non-null   object
 1   Text    3743 non-null   object
dtypes: object(2)
memory usage: 58.6+ KB


In [15]:
data.head()

,Result,Text
0,yes,I've always been very hard on myself to be thi...
1,yes,"I struggle a lot with motivation, to do someth..."
2,yes,diagnosed with dysthymia a few months ago. psy...
3,yes,I keep cycling randomly through the same half ...
4,yes,I’ve been feeling actively bad for the past fe...


from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def batch_translate_text(texts, batch_size=8):
    translated_texts = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translated_texts.extend(translated_batch)
    return translated_texts

texts = data['Text'].tolist()
translated_texts = batch_translate_text(texts)

data['Translated_Text'] = translated_texts

print(data)

In [16]:
from transformers import MarianMTModel, MarianTokenizer
import torch
from tqdm import tqdm  # Import tqdm for the progress bar

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def batch_translate_text(texts, batch_size=8):
    translated_texts = []
    # Use tqdm to create a progress bar
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translated_texts.extend(translated_batch)
    return translated_texts

# Example data
# data = pd.DataFrame({'Text': ['Text 1', 'Text 2', 'Text 3', ...]})
texts = data['Text'].tolist()
translated_texts = batch_translate_text(texts)

data['Translated_Text'] = translated_texts

print(data)


source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

c:\Users\AKAY\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AKAY\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-tc-big-en-tr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


target.spm:   0%|          | 0.00/833k [00:00<?, ?B/s]

c:\Users\AKAY\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Translating: 100%|██████████| 468/468 [4:31:54<00:00, 34.86s/it]  

     Result                                               Text  \
0       yes  I've always been very hard on myself to be thi...   
1       yes  I struggle a lot with motivation, to do someth...   
2       yes  diagnosed with dysthymia a few months ago. psy...   
3       yes  I keep cycling randomly through the same half ...   
4       yes  I’ve been feeling actively bad for the past fe...   
...     ...                                                ...   
3738    yes  my symptoms are worse than before, low confide...   
3739    yes  It gets worse when they start losing interest ...   
3740    yes                     I got diagnosed with Dysthymia   
3741    yes  I can relate to a lot of symptoms but not all ...   
3742    yes  I'm starting to think I should never feel bett...   

                                        Translated_Text  
0     Bu mükemmel ideal olmak için her zaman kendime...  
1     Motivasyonla çok mücadele ediyorum, hayatımla ...  
2     Birkaç ay önce distimi teşh

In [17]:
data

,Result,Text,Translated_Text
0,yes,I've always been very hard on myself to be thi...,Bu mükemmel ideal olmak için her zaman kendime...
1,yes,"I struggle a lot with motivation, to do someth...","Motivasyonla çok mücadele ediyorum, hayatımla ..."
2,yes,diagnosed with dysthymia a few months ago. psy...,Birkaç ay önce distimi teşhisi kondu. Psişik t...
3,yes,I keep cycling randomly through the same half ...,Aynı yarım düzine sayfa ve aramada rastgele bi...
4,yes,I’ve been feeling actively bad for the past fe...,Sadece meh hissetmek yerine son birkaç haftadı...
...,...,...,...
3738,yes,"my symptoms are worse than before, low confide...","Belirtilerim eskisinden daha kötü, düşük özgüv..."
3739,yes,It gets worse when they start losing interest ...,Sevdikleri şeylere olan ilgilerini kaybetmeye ...
3740,yes,I got diagnosed with Dysthymia,Dysthymia teşhisi kondu.
3741,yes,I can relate to a lot of symptoms but not all ...,Birçok semptomla ilişki kurabilirim ama hepsiy...


In [18]:
data.to_excel("depression_reddit_translated.xlsx", index=False)